In [24]:
import higlass
from higlass.client import View, Track, CombinedTrack
from higlass.tilesets import cooler, beddb
import os
import os.path as path
import pandas as pd
#from clodius import 

import negspy.coordinates as nc

print(os.getcwd())

/Users/hawleyj/Documents/Davos/Results/2019-10-24_higlass


In [25]:
# load annotations
chrom_labels = {
    "top": Track(
        track_type="horizontal-chromosome-labels",
        server="http://higlass.io/api/v1",
        tileset_uuid="NyITQvZsS_mOFNlz5C2LJg",
        position="top"
    ),
    "left": Track(
        track_type="vertical-chromosome-labels",
        server="http://higlass.io/api/v1",
        tileset_uuid="NyITQvZsS_mOFNlz5C2LJg",
        position="left"
    )
}

gene_annots = {
    "top": Track(
        track_type="horizontal-gene-annotations",
        server="http://higlass.io/api/v1",
        tileset_uuid="P0PLbQMwTYGy-5uPIQid7A",
        position="top"
    ),
    "left": Track(
        track_type="vertical-gene-annotations",
        server="http://higlass.io/api/v1",
        tileset_uuid="P0PLbQMwTYGy-5uPIQid7A",
        position="left"
    )
}

ctcf_motifs = {
    "top": Track(
        track_type="bedlike",
        server="http://higlass.io/api/v1",
        tileset_uuid="EkPGY0iFQx6Nq6vdF8CpWA",
        position="top"
    ),
    "left": Track(
        track_type="bedlike",
        server="http://higlass.io/api/v1",
        tileset_uuid="EkPGY0iFQx6Nq6vdF8CpWA",
        position="left"
    )
}

hg38 = nc.get_chrominfo("hg38")

In [32]:
metadata = pd.read_csv(path.join("..", "..", "Data", "External", "LowC_Samples_Data_Available.tsv"), sep="\t", header=0)
samples = ["PCa" + str(i) for i in metadata["Sample ID"].tolist()]
cooler_files = [path.join("..", "..", "Data", "Processed", "2019-06-18_PCa-LowC-sequencing", "Contacts", s + ".mcool") for s in samples]
lowc_tilesets = {s: cooler(f) for s, f in zip(samples, cooler_files)}

resolutions = [
    1000, 2000, 3000, 4000, 5000,
    10000, 20000, 30000, 40000, 50000,
    100000, 200000, 300000, 400000, 500000,
    1000000, 2000000, 3000000, 4000000, 5000000
][::-1]

lowc_heatmaps = {
    s: {
        "top": Track(
            track_type='horizontal-heatmap',
            position='top',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=150,
            options={
                "maxZoom": str(resolutions.index(10000)),
                "colorbarPosition": "topRight"
            }
        ),
        "bottom": Track(
            track_type='horizontal-heatmap',
            position='bottom',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=150,
            options={
                "maxZoom": str(resolutions.index(10000)),
                "colorbarPosition": "topRight"
            }
        ),
        "left": Track(
            track_type='vertical-heatmap',
            position='left',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            width=150,
            options={
                "maxZoom": str(resolutions.index(10000))
            }
        ),
        "right": Track(
            track_type='vertical-heatmap',
            position='right',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            width=150,
            options={
                "maxZoom": str(resolutions.index(10000))
            }
        ),
        "centre": Track(
            track_type='heatmap',
            position='center',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=500,
            options={
                "maxZoom": str(resolutions.index(10000))
            }
        )
    } for s in samples
}

In [161]:
tad_files = [path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", s + ".40000bp.aggregated-domains.bedGraph") for s in samples]
#tad_testfile = path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", "PCa3023.40000bp.aggregated-domains.bedGraph.beddb")
#tad_testfile = path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", "test.bed.beddb")
#tad_testtile = beddb(tad_testfile)
# tad_clodius_files = [path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", s + ".40000bp.aggregated-domains.bedGraph") for s in samples[4:5]]
# tad_tilesets = {s: beddb(f) for s, f in zip(samples, tad_cfiles)}
#tad_rectangles = {s: Track(
#    track_type="horizontal-2d-rectangle-domains",
#    position="top",
#    tileset=tad_tilesets[s],
#    filetype="beddb"
#) for s in samples}
#tad_testtrack = Track(
#    track_type="horizontal-2d-rectangle-domains",
#    position="top",
#    tileset=tad_testtile,
#    filetype="bed2ddb"
#)
#tad_testtrack = Track(
#    track_type="bedlike",
#    position="top",
#    tileset=tad_testtile,
#    filetype="beddb"
#)

In [12]:
foxa1_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 36000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 39000000, hg38)
    ],
)

display, server, viewconf = higlass.display([foxa1_view])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'Au-goszQSNSaeLD4SjKlaA', 'tracks': {'top': [{'ty…

In [179]:
t2e_translocation = View(
    tracks=[
        chrom_labels["top"], gene_annots["top"],
        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa13848"]["bottom"],
        lowc_heatmaps["PCa13848"]["centre"],
        lowc_heatmaps["PCa13848"]["left"],
    ],
    initialYDomain=[
        nc.chr_pos_to_genome_pos("chr21", 38000000, hg38),
        nc.chr_pos_to_genome_pos("chr21", 42000000, hg38)
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 33000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 38000000, hg38)
    ],
)
nont2e_translocation = View(
    tracks=[

        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa3023"]["top"],
        lowc_heatmaps["PCa3023"]["centre"],
        lowc_heatmaps["PCa3023"]["left"],
    ],
    initialYDomain=[
        nc.chr_pos_to_genome_pos("chr21", 38000000, hg38),
        nc.chr_pos_to_genome_pos("chr21", 42000000, hg38)
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 33000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 38000000, hg38)
    ],
)

display, server, viewconf = higlass.display([t2e_translocation, nont2e_translocation])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'fkRiwNlsSGy449nV3nm0pQ', 'tracks': {'top': [{'ty…

In [43]:
ar_tracks = [chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples]
ar_view = View(
    tracks=ar_tracks,
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chrX", 65040000, hg38),
        nc.chr_pos_to_genome_pos("chrX", 70200000, hg38)
    ]
)

display, server, viewconf = higlass.display([ar_view], value_scale_syncs = [ar_tracks])
display
#help(higlass.display)

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'Cdg-ozpSQUyoZ-v5Z3ibug', 'tracks': {'top': [{'ty…

In [18]:
myc_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr8", 126000000, hg38),
        nc.chr_pos_to_genome_pos("chr8", 130000000, hg38)
    ],
)

display, server, viewconf = higlass.display([myc_view])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'Ltt4yD7PSGu8sWAiQcZ5fw', 'tracks': {'top': [{'ty…

In [23]:
ctcf_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr16", 65000000, hg38),
        nc.chr_pos_to_genome_pos("chr16", 69000000, hg38)
    ],
)
yy1_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 98000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 102000000, hg38)
    ],
)
hoxb13_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr17", 46160000, hg38),
        nc.chr_pos_to_genome_pos("chr17", 51320000, hg38)
    ],
)

display, server, viewconf = higlass.display([
    ctcf_view
])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'aEyN0ixJTzCXiOO_MrbBPQ', 'tracks': {'top': [{'ty…

In [42]:
t2e_view = View(
    tracks=(
        [chrom_labels["top"], gene_annots["top"]]
        + [lowc_heatmaps["PCa" + str(s)]["top"] for s in metadata.loc[metadata["T2E Status"] == "Yes", "Sample ID"]]
        + [lowc_heatmaps["PCa" + str(s)]["top"] for s in metadata.loc[metadata["T2E Status"] == "No", "Sample ID"]]
    ),
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr21", 30000000, hg38),
        nc.chr_pos_to_genome_pos("chr21", 48000000, hg38)
    ],
)

display, server, viewconf = higlass.display([
    t2e_view
])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'E6czkXM5RUus3KW_CQF1HA', 'tracks': {'top': [{'ty…